In [2]:
import sys
sys.path.append('..')

from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from enum import Enum

from src.services import facebook_creative_descriptive_saver as fcd
from src.models.enums.facebook import Target
from src.database.session import SessionLocal
from src.pingers import *
from src.utils import *
from src import crud
from src.crud.utils.get_performance import get_performance, get_conversion_rates_dict
db = SessionLocal()
end_date_plus_one = date(year=2023, month=8, day=1)
end_date = end_date_plus_one - timedelta(days=1)
start_date = end_date_plus_one - relativedelta(months=24)
shop_id = 2445081

2023-08-21 14:33:26.029 | DEBUG    | src.database.session:<module>:17 - base: prod.cw6tznt1p9hq.us-east-1.rds.amazonaws.com


In [56]:
class Metric(str, Enum):
    cpm='cpm'
    ctr='ctr'
    cr='cr'
    atc='atc'
    patc='patc'

metric_formula_dict = {
    Metric.cpm: (lambda df: 1000 * df.spend_USD / df.impr if df.impr > 0 else np.nan),
    Metric.ctr: (lambda df: df.clicks / df.impr if df.impr > 0 else np.nan),
    Metric.cr: (lambda df: df.purch / df.clicks if df.clicks > 0 else np.nan),
    # Metric.atc: lambda df: df.add_to_cart / df.clicks if df.clicks > 0 else np.nan,
    # Metric.patc: lambda df: df.purch / df.add_to_cart if df.add_to_cart > 0 else np.nan,
}

main_df = fcd.read_df('main', end_date=end_date)
for metric, formula in metric_formula_dict.items():
    main_df[metric] = main_df.apply(formula, axis=1)
feature = 'emoji'
feature_df = main_df[(main_df.feature == feature)]
feature_df

# metric_format_dict = {
#     Metric.cpm: lambda x, pos: f"${round(x)}",
#     Metric.ctr: lambda x, pos: f"{round(x*100, 2)}%",
#     Metric.cr: lambda x, pos: f"{round(x*100, 2)}%",
#     Metric.atc: lambda x, pos: f"{round(x*100, 2)}%",
#     Metric.patc: lambda x, pos: f"{round(x*100, 2)}%",
# }

In [61]:
main_df = fcd.read_df('main', end_date=end_date)
for metric, formula in metric_formula_dict.items():
    main_df[metric] = main_df.apply(formula, axis=1)
feature = 'emoji'
feature_df = main_df[(main_df.feature == feature)]
feature_df

,year_month,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,feature,shop_id,Metric.cpm,Metric.ctr,Metric.cr
15,2021-08,False,68653.76,5438835.0,12168.0,1076.0,51218.45,126,emoji,2,12.622880,0.002237,0.088429
16,2021-08,True,63992.60,3871140.0,13294.0,906.0,34649.13,82,emoji,2,16.530686,0.003434,0.068151
17,2022-06,True,626.67,27796.0,281.0,0.0,0.00,8,emoji,2,22.545330,0.010109,0.000000
18,2022-07,True,1182.74,102443.0,251.0,0.0,0.00,5,emoji,2,11.545347,0.002450,0.000000
19,2022-11,True,547.01,47829.0,2565.0,0.0,0.00,3,emoji,2,11.436785,0.053629,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179458,2023-05,True,9111.60,756332.0,5551.0,326.0,39565.05,44,emoji,45514705,12.047090,0.007339,0.058728
179459,2023-06,False,1781.21,118512.0,462.0,72.0,9240.85,9,emoji,45514705,15.029786,0.003898,0.155844
179460,2023-06,True,8068.54,610407.0,3636.0,231.0,29074.56,32,emoji,45514705,13.218295,0.005957,0.063531
179461,2023-07,False,1097.14,52291.0,236.0,54.0,7083.32,9,emoji,45514705,20.981431,0.004513,0.228814


In [58]:
main_df.apply(lambda df: 1000 * df.spend_USD / df.impr if df.impr > 0 else np.nan, axis=1)

0        14.247768
1        22.545330
2        11.545347
3        11.436785
4        14.453500
           ...    
92564     5.393949
92565     2.667455
92566     8.539192
92567    12.300588
92568     6.209309
Length: 92569, dtype: float64

In [60]:
feature = 'emoji'
value = True
feature_df = main_df[(main_df.feature == feature)]
feature_df

,year_month,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,feature,shop_id,Metric.cpm,Metric.ctr,Metric.cr
15,2021-08,False,68653.760000,5438835.0,12168.0,1076.0,51218.450000,126,emoji,2,12.622880,0.002237,0.088429
16,2021-08,True,63992.600000,3871140.0,13294.0,906.0,34649.130000,82,emoji,2,16.530686,0.003434,0.068151
17,2022-06,True,626.670000,27796.0,281.0,0.0,0.000000,8,emoji,2,22.545330,0.010109,0.000000
18,2022-07,True,1182.740000,102443.0,251.0,0.0,0.000000,5,emoji,2,11.545347,0.002450,0.000000
19,2022-11,True,547.010000,47829.0,2565.0,0.0,0.000000,3,emoji,2,11.436785,0.053629,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92344,2022-12,False,572.190157,106080.0,603.0,9.0,886.806633,1,emoji,33212206,5.393949,0.005684,0.014925
92345,2023-01,False,30.283622,11353.0,47.0,0.0,0.000000,1,emoji,33212206,2.667455,0.004140,0.000000
92346,2023-05,False,1628.466605,190705.0,777.0,15.0,1017.514892,12,emoji,33212206,8.539192,0.004074,0.019305
92347,2023-06,False,199.220326,16196.0,58.0,2.0,107.977032,1,emoji,33212206,12.300588,0.003581,0.034483


In [54]:
for metric, formula in metric_formula_dict.items():
    main_df[metric] = main_df.apply(formula, axis=1)
feature = 'emoji'
feature_df = main_df[(main_df.feature == feature)]
feature_df

In [19]:
formula(main_df.loc[0, :])

TypeError: <lambda>() missing 1 required positional argument: 'pos'

In [2]:
fcd.create_and_save_summary(end_date=end_date)

2023-08-21 12:49:52.905 | DEBUG    | src.services.descriptive.descriptive_saver:create_and_save_summary:36 - ['year_month', 'feature', 'feature_value']


In [2]:
summary_df = fcd.read_df('summary', end_date=end_date)
summary_df

,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,n_shops,spend_USD_by_shop,impr_by_shop,clicks_by_shop,purch_by_shop,purch_value_USD_by_shop,n_ads_by_shop
0,2021-08,creative_type,carousel,42030.279927,2720853.0,33467.0,502.0,90318.793686,45,2,0.794254,0.678537,0.782195,0.738235,0.796761,0.398230
1,2021-08,creative_type,dynamic,10887.664708,1289029.0,9319.0,178.0,23038.652777,68,2,0.205746,0.321463,0.217805,0.261765,0.203239,0.601770
2,2021-08,cta,False,52917.944635,4009882.0,42786.0,680.0,113357.446463,113,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,2021-08,discount,False,35690.936214,2774374.0,29747.0,455.0,78983.838743,39,2,0.674458,0.691884,0.695251,0.669118,0.696768,0.345133
4,2021-08,discount,True,17227.008421,1235508.0,13039.0,225.0,34373.607720,74,2,0.325542,0.308116,0.304749,0.330882,0.303232,0.654867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,2023-07,urgency,False,259475.009953,17869595.0,175611.0,3166.0,477205.144986,69,2,0.971241,0.926095,0.955399,0.881894,0.939683,0.945205
712,2023-07,urgency,True,7683.105525,1426041.0,8198.0,424.0,30631.421755,4,1,0.028759,0.073905,0.044601,0.118106,0.060317,0.054795
713,2023-07,user_addressing,False,9096.610000,598525.0,4972.0,42.0,7534.200000,3,1,0.034050,0.031019,0.027050,0.011699,0.014836,0.041096
714,2023-07,user_addressing,True,258061.505478,18697111.0,178837.0,3548.0,500302.366742,70,2,0.965950,0.968981,0.972950,0.988301,0.985164,0.958904


In [5]:
summary_df[summary_df.feature == 'creative_type'].groupby(['year_month', 'feature']).sum()

,,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,n_shops,spend_USD_by_shop,impr_by_shop,clicks_by_shop,purch_by_shop,purch_value_USD_by_shop,n_ads_by_shop
year_month,feature,,,,,,,,,,,,,,
2021-08,creative_type,carouseldynamic,52917.944635,4009882.0,42786.0,680.0,113357.446463,113,4,1.0,1.0,1.0,1.0,1.0,1.0
2021-09,creative_type,carouseldynamic,24165.890637,1971449.0,17117.0,630.0,63470.895325,66,3,1.0,1.0,1.0,1.0,1.0,1.0
2021-10,creative_type,carouseldynamic,26427.459235,2223481.0,20100.0,824.0,87163.549962,82,3,1.0,1.0,1.0,1.0,1.0,1.0
2021-11,creative_type,carouseldynamic,52778.190299,3587435.0,31512.0,2046.0,237975.227778,98,4,1.0,1.0,1.0,1.0,1.0,1.0
2021-12,creative_type,carouseldynamicimageunknown,46631.926584,3184183.0,35885.0,876.0,105553.249608,178,6,1.0,1.0,1.0,1.0,1.0,1.0
2022-01,creative_type,carouseldynamicimageunknownvideo,137052.389716,9453957.0,102873.0,1268.0,239731.618990,121,7,1.0,1.0,1.0,1.0,1.0,1.0
2022-02,creative_type,carouseldynamicimagevideo,195549.138820,9303403.0,90777.0,1487.0,288846.630864,65,7,1.0,1.0,1.0,1.0,1.0,1.0
2022-03,creative_type,carouseldynamicimagevideo,277006.652355,14503907.0,134153.0,4358.0,598468.803013,312,7,1.0,1.0,1.0,1.0,1.0,1.0
2022-04,creative_type,carouseldynamicimagevideo,89604.532922,3828222.0,42996.0,1005.0,168579.315022,165,7,1.0,1.0,1.0,1.0,1.0,1.0


In [4]:
main_df = fcd.get_main_df(end_date=end_date)
main_df

spend_USD         impr  \
year_month feature         feature_value                              
2021-08    creative_type   carousel       1.385117e+06  189679520.0   
                           dynamic        2.015207e+06  231822297.0   
                           image          1.122925e+06  179716465.0   
                           unknown        1.297635e+06  193497346.0   
                           video          1.305978e+06   95865870.0   
...                                                ...          ...   
2023-07    urgency         True           6.187278e+05  110600249.0   
           user_addressing False          1.931415e+06  371170936.0   
                           True           3.717662e+06  418116014.0   
           weasel_words    False          4.883056e+06  714306181.0   
                           True           7.660223e+05   74980769.0   

                                             clicks     purch  \
year_month feature         feature_value                        
2021-08    creative_type   carousel       2731351.0   36151.0   
                           dynamic        2174472.0   41599.0   
                           image          1288717.0   22366.0   
                           unknown        2302229.0   40817.0   
                           video          1101071.0   31390.0   
...                                             ...       ...   
2023-07    urgency         True           1707297.0   30406.0   
           user_addressing False          4285518.0   67128.0   
                           True           6660266.0  112665.0   
           weasel_words    False          9943996.0  161682.0   
                           True           1001788.0   18111.0   

                                          purch_value_USD    n_ads  \
year_month feature         feature_value                             
2021-08    creative_type   carousel          2.932261e+06   4929.0   
                           dynamic           2.836585e+06   9757.0   
                           image             1.323270e+06   7237.0   
                           unknown           2.087038e+06   8565.0   
                           video             1.556321e+06   4154.0   
...                                                   ...      ...   
2023-07    urgency         True              1.747109e+06   2382.0   
           user_addressing False             5.646147e+06   6168.0   
                           True              9.267490e+06  10101.0   
           weasel_words    False             1.368683e+07  13919.0   
                           True              1.226810e+06   2350.0   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2021-08    creative_type   carousel              137.795773    131.217613   
                           dynamic               122.230413    126.040613   
                           image                  90.810532     91.908048   
                           unknown               118.337687    122.586704   
                           video                  53.825595     52.247022   
...                                                     ...           ...   
2023-07    urgency         True                   53.511801     51.176310   
           user_addressing False                 220.288345    224.220472   
                           True                  209.711655    205.779528   
           weasel_words    False                 387.141526    385.659054   
                           True                   42.858474     44.340946   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2021-08    creative_type   carousel           133.292906     130.363844   
                           dynamic            122.505200     103.078691   
                           image               91.310972      

In [5]:
set(main_df.index.get_level_values(level='feature').tolist())

{'creative_type',
 'cta',
 'discount',
 'emoji',
 'fact_words',
 'fake_column_for_total_sum',
 'free_shipping',
 'hashtag',
 'link',
 'percentage',
 'price',
 'starts_with_question',
 'target',
 'urgency',
 'user_addressing',
 'weasel_words'}

In [6]:
main_df.reset_index(inplace=True)

In [8]:
df_fw = fcd.get_shop_descriptive_df(db=db, shop_id=16038, end_date=end_date, start_date = start_date)
df_dr = fcd.get_shop_descriptive_df(db=db, shop_id=44301396, end_date=end_date, start_date = start_date)
df_test = df_fw.add(df_dr, fill_value=0)
df_test

2023-08-17 11:13:28.909 | DEBUG    | src.services.descriptive.descriptive_saver:get_shop_descriptive_df:115 - 16038: +
2023-08-17 11:13:34.219 | DEBUG    | src.services.descriptive.descriptive_saver:get_shop_descriptive_df:115 - 44301396: +


spend_USD        impr    clicks  \
year_month feature         feature_value                                        
2021-08    creative_type   carousel        42026.960098   2720853.0   33467.0   
                           dynamic         10883.048792   1289029.0    9319.0   
           cta             False           52910.008889   4009882.0   42786.0   
           discount        False           35686.750467   2774374.0   29747.0   
                           True            17223.258422   1235508.0   13039.0   
...                                                 ...         ...       ...   
2023-07    urgency         False          259459.083828  17869595.0  175611.0   
                           True             7673.135233   1426041.0    8198.0   
           user_addressing False            9096.610000    598525.0    4972.0   
                           True           258035.609061  18697111.0  178837.0   
           weasel_words    False          267132.219061  19295636.0  183809.0   

                                           purch  purch_value_USD  n_ads  \
year_month feature         feature_value                                   
2021-08    creative_type   carousel        502.0     90307.824811   45.0   
                           dynamic         178.0     23024.805496   68.0   
           cta             False           680.0    113332.630308  113.0   
           discount        False           455.0     78971.516448   39.0   
                           True            225.0     34361.113859   74.0   
...                                          ...              ...    ...   
2023-07    urgency         False          3166.0    477140.981293   69.0   
                           True            424.0     30591.671656    4.0   
           user_addressing False            42.0      7534.200000    3.0   
                           True           3548.0    500198.452949   70.0   
           weasel_words    False          3590.0    507732.652949   73.0   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2021-08    creative_type   carousel                1.261710      1.180138   
                           dynamic                 0.738290      0.819862   
           cta             False                   2.000000      2.000000   
           discount        False                   1.221120      1.279078   
                           True                    0.778880      0.720922   
...                                                     ...           ...   
2023-07    urgency         False                   1.614993      1.602622   
                           True                    0.385007      0.397378   
           user_addressing False                   0.036798      0.038106   
                           True                    1.963202      1.961894   
           weasel_words    False                   2.000000      2.000000   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2021-08    creative_type   carousel             1.273955       1.291995   
                           dynamic              0.726045       0.708005   
           cta             False                2.000000       2.000000   
           discount        False                1.248224       1.260297   
                           True                 0.751776       0.739703   
...                                                  ...            ...   
2023-07    urgency         False                1.657332       1.648425   
                           True                 0.342668       0.351575   
           user_addressing False                0.031097       0.017617   
                           True                 1.968903       1.982383   
           weasel_words    False                2.000000       2.000000   

                         

In [9]:
main_df.subtract(df_test)

spend_USD  impr  clicks  purch  \
year_month feature         feature_value                                   
2021-08    creative_type   carousel             0.0   0.0     0.0    0.0   
                           dynamic              0.0   0.0     0.0    0.0   
           cta             False                0.0   0.0     0.0    0.0   
           discount        False                0.0   0.0     0.0    0.0   
                           True                 0.0   0.0     0.0    0.0   
...                                             ...   ...     ...    ...   
2023-07    urgency         False                0.0   0.0     0.0    0.0   
                           True                 0.0   0.0     0.0    0.0   
           user_addressing False                0.0   0.0     0.0    0.0   
                           True                 0.0   0.0     0.0    0.0   
           weasel_words    False                0.0   0.0     0.0    0.0   

                                          purch_value_USD  n_ads  \
year_month feature         feature_value                           
2021-08    creative_type   carousel          0.000000e+00    0.0   
                           dynamic           0.000000e+00    0.0   
           cta             False             0.000000e+00    0.0   
           discount        False             0.000000e+00    0.0   
                           True              0.000000e+00    0.0   
...                                                   ...    ...   
2023-07    urgency         False             0.000000e+00    0.0   
                           True              0.000000e+00    0.0   
           user_addressing False             0.000000e+00    0.0   
                           True              5.820766e-11    0.0   
           weasel_words    False             0.000000e+00    0.0   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2021-08    creative_type   carousel            0.000000e+00  0.000000e+00   
                           dynamic             0.000000e+00  0.000000e+00   
           cta             False               0.000000e+00  0.000000e+00   
           discount        False               0.000000e+00  2.220446e-16   
                           True                0.000000e+00  0.000000e+00   
...                                                     ...           ...   
2023-07    urgency         False               0.000000e+00  0.000000e+00   
                           True                0.000000e+00  0.000000e+00   
           user_addressing False              -2.081668e-17 -1.387779e-17   
                           True                0.000000e+00  0.000000e+00   
           weasel_words    False               0.000000e+00  0.000000e+00   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2021-08    creative_type   carousel        -2.220446e-16   0.000000e+00   
                           dynamic          0.000000e+00   0.000000e+00   
           cta             False            0.000000e+00   0.000000e+00   
           discount        False            0.000000e+00   0.000000e+00   
                           True             0.000000e+00   0.000000e+00   
...                                                  ...            ...   
2023-07    urgency         False            0.000000e+00   0.000000e+00   
                           True            -5.551115e-17   0.000000e+00   
           user_addressing False           -3.122502e-17  -2.775558e-17   
                           True            -2.220446e-16   0.000000e+00   
           weasel_words    False            0.000000e+00   0.000000e+00   

                                          purch_value_USD_by_shop  \
year_month feature         feature_value                            
2021-08    creative_type   carousel                 